## 안전관리비 현황 수집

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "안전관리비 현황"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/안전관리비 현황/안전관리비 현황.csv'

In [2]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectSftMctList'

In [3]:
APIKEYLEN = len(APIKEY)

## 연도별 공사계약 목록에서  SPTNO 활용

In [4]:
#임시 
imsiDf = pd.read_csv("C:/Users/luthe/Desktop/python_automation-master/apiScrapy/output/건설사업정보시스템/연도별 공사계약 목록/연도별 공사계약 목록.csv", encoding="ms949")

In [5]:
dfCombi = imsiDf.loc[:,["sptno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

1220

## 입력 파라미터 정보 불러오기

In [6]:
newParam = dfCombi.sptno.drop_duplicates().tolist()

## 파라미터 정보 불러오기

In [7]:
orgParam = []

try:
    # 기존 데이터 및 파라미터 정보 불러오기
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

sptnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if sptnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output\\건설사업정보시스템\\안전관리비 현황\\selectSftMctList.pickle'
안전관리비 현황 정보 new 모드 2 


In [8]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
PRIMARYKEY

'serviceKey,sptNo,crsdYm'

In [9]:
JSONKEY = "detail1"
DUMMY = 0

In [10]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
PAGEYN

0

In [11]:
g_totalCount = 0

## 안전관리비 현황 전용 스크랩 함수

In [12]:
# import requests
# resp = requests.get("https://www.calspia.go.kr/io/openapi/cm/selectSftMctList.do?serviceKey=9D486CED-81E4-4884-96FF-BB1F14A9223B&sptNo=C1998011&crsdYm=&type=json")

In [13]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            time.sleep(1)
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            print(inUrl+queryParams)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                        
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                emptyPd = emptyPd.append(pd.DataFrame([response.json()["response"]["body"]["detail1"]]))

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"]["items"]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        # print(rowData)
                        emptyPd = pd.merge(left = emptyPd , right = rowData, on = "sptNo")
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [14]:
numOfRows = 1000 # 수행 성능향상을 위해 100개씩 요청하는 것으로 수정해야 함 10 -> 1000, BASEPARAM에도 추가해야 함
PAGEYN=0

## 테스트용 코드

In [15]:
resultDf = pd.DataFrame()
resultDfMerged = pd.DataFrame()

APICALL = 0 #추가
sptnoListLen = len(sptnoList)
pageList = [] #추가

for i in range(0,sptnoListLen):
    pageNo = 1
    global g_totalCount
    g_totalCount = 0
    sptnoValue = sptnoList[i]
    apiNo = i%APIKEYLEN
    BASEPARAM={"serviceKey":APIKEY[apiNo],"sptNo": sptnoValue, "crsdYm": "", "type":"json"}
    print("BASEPARAM ",BASEPARAM)
    resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
    print(type(resultDf))
    print("i= ",i)
    print("resultDf ",resultDf)
    if resultDf is None or resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
    else:
        dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
        for i in range(0,len(dateRange)):
            apiNo = i%APIKEYLEN
            BASEPARAM={"serviceKey":APIKEY[apiNo],"sptNo": sptnoValue, "crsdYm": str(dateRange[i]), "type":"json"}
            print("BASEPARAM ",BASEPARAM)
            resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
            resultDfMerged = resultDfMerged.append(resultDf) 


BASEPARAM  {'serviceKey': '9D486CED-81E4-4884-96FF-BB1F14A9223B', 'sptNo': 'C1998019', 'crsdYm': '', 'type': 'json'}
1 page scraping start
https://www.calspia.go.kr/io/openapi/cm/selectSftMctList.do?serviceKey=9D486CED-81E4-4884-96FF-BB1F14A9223B&sptNo=C1998019&crsdYm=&type=json
>totalCount 9
   seq     sptNo sftMctItmCd             cmnDtlCdNm  bExeAmt  exeAmt  sExeAmt  \
0    1  C1998019           1  1.안전보건관계자의 인건비 및 업무수당        0       0        0   
1    1  C1998019           2             2.안전보건 시설비        0       0        0   
2    1  C1998019           3            3.개인보호구 구입비        0       0        0   
3    1  C1998019           4             4.사업장안전진단비        0       0        0   
4    1  C1998019           5        5.안전보건 교육 및 행사비        0       0        0   
5    1  C1998019           6            6.근로자 건강진단비        0       0        0   
6    1  C1998019           7        7.건설재해예방 기술 지도비        0       0        0   
7    1  C1998019           8               8.본사 관리비       

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(requests.get("https://www.calspia.go.kr/io/openapi/cm/selectSftMctList.do?serviceKey=9D486CED-81E4-4884-96FF-BB1F14A9223B&sptNo=C2005032&crsdYm=&type=json").json()["response"]["body"]["items"])

In [ ]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME, mode)

In [ ]:
resultDfMerged.shape

In [ ]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
sptnoList = orgParam + sptnoList

# 최종 파라미터 저장
cf.saveparam(sptnoList, SITENAME,DATANAME,SERVICENAME)